In [2]:
import os
import pandas as pd
import numpy as np
from textblob import TextBlob


In [3]:
os.chdir('../scripts/')
import data_loader as data_loader

In [4]:
all_data_path = '../data/raw_analyst_ratings.csv'
df = data_loader.read_csv_file(all_data_path)

In [5]:
df = df.get("data")
df.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


In [6]:
df['date'] = pd.to_datetime(df['date'], format='ISO8601')
df.head()

,headline,url,publisher,date,stock
0,Stocks That Hit 52-Week Highs On Friday,https://www.benzinga.com/news/20/06/16190091/s...,Benzinga Insights,2020-06-05 10:30:54-04:00,A
1,Stocks That Hit 52-Week Highs On Wednesday,https://www.benzinga.com/news/20/06/16170189/s...,Benzinga Insights,2020-06-03 10:45:20-04:00,A
2,71 Biggest Movers From Friday,https://www.benzinga.com/news/20/05/16103463/7...,Lisa Levin,2020-05-26 04:30:07-04:00,A
3,46 Stocks Moving In Friday's Mid-Day Session,https://www.benzinga.com/news/20/05/16095921/4...,Lisa Levin,2020-05-22 12:45:06-04:00,A
4,B of A Securities Maintains Neutral on Agilent...,https://www.benzinga.com/news/20/05/16095304/b...,Vick Meyer,2020-05-22 11:38:59-04:00,A


# Correlation for Amazon

In [45]:
df['stock'].unique()
grouped = df.groupby('stock')

# Access the group for AMZN
amzn_group = grouped.get_group('AMZN')



In [33]:
all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df2 = data_loader.read_csv_file(all_data_path)
df2 = df2.get("data")
df2['Date'] = pd.to_datetime(df2['Date'], format='ISO8601')

df2.head()
df2.shape

(6846, 9)

In [46]:
# Create a DataFrame with dates from the AMZN data
dates = df2.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [37]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(amzn_group['headline'], size=len(text_data))

# Display the first few rows of the updated DataFrame
text_data.head()

,Date,Headline
0,0,Making Sense Of Why Consumers Are Switching Th...
1,1,Amazon Expects To Invest $4B Apr.-June On Covi...
2,2,AMC Shares Largely Unaffected As Traders Circu...
3,3,"Locked Down, Demand Up: Amazon Shines As Virus..."
4,4,IRS Alerts Against Criminals Profiteering From...


In [38]:
def calculate_sentiment(text):
    return TextBlob(text).sentiment.polarity

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [39]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()

,Headline,Sentiment,Close,Daily Returns
Date,,,,
0,Making Sense Of Why Consumers Are Switching Th...,0.000000,0.097917,NaN
1,Amazon Expects To Invest $4B Apr.-June On Covi...,0.000000,0.086458,-0.117028
2,AMC Shares Largely Unaffected As Traders Circu...,-0.050000,0.085417,-0.012040
3,"Locked Down, Demand Up: Amazon Shines As Virus...",-0.155556,0.081771,-0.042685
4,IRS Alerts Against Criminals Profiteering From...,-0.208333,0.071354,-0.127392


In [40]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)


Correlation between sentiment and daily stock returns: -0.005804684386003446


In [41]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

Correlation between sentiment and closing: 0.009027455325930045


# Calculate the correlation for GOOGLE


In [31]:
goog_group = grouped.get_group('GOOG')

In [43]:
all_data_path = '../data/yfinance_data/GOOG_historical_data.csv'
df3 = data_loader.read_csv_file(all_data_path)
df3 = df3.get("data")
df3['Date'] = pd.to_datetime(df2['Date'], format='ISO8601')

df3.head()
df3.shape

(5020, 9)

In [44]:
# Create a DataFrame with dates from the GOOGL data
dates = df3.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [ ]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(goog_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()

In [ ]:

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [ ]:
combined_data = text_data.set_index('Date').join(df3['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


In [ ]:

combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

In [ ]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

# correlation on META

meta_group = grouped.get_group('META')

In [ ]:

all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df4 = data_loader.read_csv_file(all_data_path)
df4 = df4.get("data")
df4['Date'] = pd.to_datetime(df4['Date'], format='ISO8601')
df4.head()
df4.shape

In [ ]:
# Create a DataFrame with dates from the AAPL data
dates = df4.index
text_data = pd.DataFrame(dates, columns=['Date'])


In [ ]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(meta_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()

In [ ]:

# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)




In [ ]:
combined_data = text_data.set_index('Date').join(df4['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()

In [ ]:

combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

In [ ]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

# Correlation on Nvidia

nvda_group = grouped.get_group('NVDA')

In [ ]:
all_data_path = '../data/yfinance_data/NVDA_historical_data.csv'
df5 = data_loader.read_csv_file(all_data_path)
df5 = df5.get("data")
df5['Date'] = pd.to_datetime(df5['Date'], format='ISO8601')
df5.head()
df5.shape

In [ ]:

# Create a DataFrame with dates from the AAPL data
dates = df5.index
text_data = pd.DataFrame(dates, columns=['Date'])

In [ ]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(nvda_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()


In [ ]:
# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [ ]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


In [ ]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

In [ ]:
correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)

# Correlation on Tesla

tesla_group = grouped.get_group('TSLA')


In [ ]:
all_data_path = '../data/yfinance_data/AMZN_historical_data.csv'
df6 = data_loader.read_csv_file(all_data_path)
df6 = df6.get("data")
df6['Date'] = pd.to_datetime(df6['Date'], format='ISO8601')
df6.head()
df6.shape

In [ ]:
# Create a DataFrame with dates from the AAPL data
dates = df6.index
text_data = pd.DataFrame(dates, columns=['Date'])

In [ ]:
# Assign a random headline to each date
text_data['Headline'] = np.random.choice(tsla_group['headline'], size=len(text_data))
# Display the first few rows of the updated DataFrame
text_data.head()


In [ ]:
# Applying the sentiment analysis function to your headlines
text_data['Sentiment'] = text_data['Headline'].apply(calculate_sentiment)

In [ ]:
combined_data = text_data.set_index('Date').join(df2['Close'])
combined_data.head()
combined_data['Daily Returns'] = combined_data['Close'].pct_change()
combined_data.head()


In [ ]:
combined_data.dropna(inplace=True)
correlation = combined_data['Sentiment'].corr(combined_data['Daily Returns'])
print("Correlation between sentiment and daily stock returns:", correlation)

In [ ]:

correlation = combined_data['Sentiment'].corr(combined_data['Close'])
print("Correlation between sentiment and closing:", correlation)
